In [2]:
using CairoMakie, Dierckx, Optim, LinearAlgebra, QuantEcon

In [3]:
function create_EGM_model(;nk = 101, nz = 19)

    p = (;α = 0.33, # Capital share
    β = 0.96, # Discount factor
    A = 10.0, # TFP
    γ = 2.0, # Risk aversion
    δ = 0.1, # Depreciation rate
    nk = nk, # Number of capital gridpoints
    nz = nz, # Number of shock gridpoints
    θ = 2, # Grid expansion parameter
    lb = 10^-4, # Lower bound of capital grid
    ub = 100.0, # Upper bound of capital grid
    ρ = 0.98, # Persistence of AR(1) process
    μ = 0.5, # Mean of AR(1) process
    σ = 0.01, # Var of AR(1) process
    toler = 1e-6, # Tolerance
    maxiter = 1_000, # Maximum number of iterations
    print_skip = 10, # Print every x iterations
    kgrid = LinRange(0.0,1.0,nk), # Capital grid
    zgrid = LinRange(0.0,1.0,nz), # Shock grid
    Π = zeros(Float64, nz, nz)) # Transition matrix

    # Create productivity grid and transition matrix
    mc = rouwenhorst(p.nz, p.μ, p.ρ, p.σ)
    zgrid, Π = exp.(mc.state_values), mc.p

    # Create capital grid
    temp_grid = LinRange(0.0,1.0,p.nk)
    kgrid = p.lb .+ (p.ub - p.lb) * (temp_grid .^ p.θ)

    # Return NamedTuple
    p = (p..., kgrid = kgrid, zgrid = zgrid, Π = Π)

    return p
end

model_EGM = create_EGM_model()

(α = 0.33, β = 0.96, A = 10.0, γ = 2.0, δ = 0.1, nk = 101, nz = 19, θ = 2, lb = 0.0001, ub = 100.0, ρ = 0.98, μ = 0.5, σ = 0.01, toler = 1.0e-6, maxiter = 1000, print_skip = 10, kgrid = [0.0001, 0.01009999, 0.040099960000000004, 0.09009990999999999, 0.16009984, 0.25009975000000007, 0.36009963999999994, 0.49009951, 0.64009936, 0.81009919  …  82.81001719000001, 84.64001536, 86.49001351000001, 88.36001164, 90.25000975, 92.16000783999999, 94.09000591, 96.04000395999999, 98.01000198999999, 100.0], zgrid = [0.008387608038651633, 0.014299146011397482, 0.024377101995354325, 0.041557943475662926, 0.07084774335585627, 0.12078082597029194, 0.20590645842864538, 0.35102814773808405, 0.5984307701894207, 1.0202013400267553, 1.7392333851131028, 2.9650350859298533, 5.054774785284692, 8.617351022656463, 14.690810531036542, 25.044809418971962, 42.696247256574644, 72.78831710388745, 124.0891049505581], Π = [0.005637710113660432 0.033826260681962594 … 7.8580342233181e-10 1.4551915228366852e-11; 0.001879236

In [8]:
function utility(c)
    if model_EGM.γ == 1.0
        return log(c)
    else
        return (c^(1-model_EGM.γ)-1) / (1-model_EGM.γ)
    end
end 

function marginal_utility(c)
    return c^(-model_EGM.γ)
end

function inverse_marginal_utility(u)
    return u^(-1/model_EGM.γ)
end

function resources(i ,j , p)
    (; α, δ, A, kgrid, zgrid) = p
    return (A * zgrid[j] * (kgrid[i] ^ α)) + (1.0 - δ) * kgrid[i]
end

function marginal_prod(i ,j , p)
    (; α, δ, A, kgrid, zgrid) = p
    return α * A * zgrid[j] * (kgrid[i] ^ (α - 1)) + (1.0 - δ)
end

function initial_guess(p)
    (; nk, nz) = p
    c_pol = zeros(nk, nz)
    for i in 1:nk
        for j in 1:nz
            c_pol[i,j] = resources(i ,j , p)
        end
    end
    return c_pol
end

function resource_grid(p)
    (; nk, nz) = p
    Ygrid = zeros(nk, nz)
    for i in 1:nk
        for j in 1:nz
            Ygrid[i,j] = resources(i, j, p)
        end
    end
    return Ygrid
end

function marginal_prod_grid(p)
    (; nk, nz) = p
    MPgrid = zeros(nk, nz)
    for i in 1:nk
        for j in 1:nz
            MPgrid[i,j] = marginal_prod(i, j, p)
        end
    end
    return MPgrid
end

marginal_prod_grid (generic function with 1 method)

In [11]:
function new_egm(p)
    # Unpack parameters
    (; print_skip, toler, maxiter, Π, kgrid, zgrid, nk, nz, β) = p

    # Initialise matrices
    c_pol_1 = initial_guess(p)
    c_pol_2 = zeros(nk, nz)
    s_pol = zeros(nk, nz)
    V1 = zeros(nk, nz)
    V2 = zeros(nk, nz)
    V3 = zeros(nk, nz)
    V4 = zeros(nk, nz)
    v_out = zeros(nk, nz)
    MPgrid = marginal_prod_grid(p)
    Ygrid = resource_grid(p)
 
    # Set error and iteration counter
    error = toler + 1
    iter = 0
    if iter == 0
        println("Iterating...")
    end

    while error > toler && iter < maxiter
        MUgrid = marginal_utility.(c_pol_1)
        V1 = β .* MUgrid .* MPgrid
        V2 = (Π * V1')'
        V3 = inverse_marginal_utility.(V2)
        V4 = kgrid .+ V3
        for j in 1:nz
            for i in 1:nk
                opt = optimize(k -> A * zgrid[j] * (k ^ α) + (1.0 - δ) * k - V4[i,j], autodiff = :forward)
                s_pol[i,j] = opt.minimizer
            end
        end
    end
    return s_pol
end

test = new_egm(model_EGM)

Iterating...


MethodError: MethodError: no method matching optimize(::var"#18#19"{Vector{Float64}, Int64, Int64}; autodiff::Symbol)

Closest candidates are:
  optimize(::Any, !Matched::Number, !Matched::AbstractArray, !Matched::AbstractArray{T}, !Matched::Optim.AbstractConstrainedOptimizer, !Matched::Optim.Options; kwargs...) where T
   @ Optim C:\Users\IRCSB01\.julia\packages\Optim\ZhuZN\src\multivariate\solvers\constrained\fminbox.jl:253
  optimize(::Any, !Matched::Number, !Matched::AbstractArray, !Matched::AbstractArray{T}, !Matched::Optim.AbstractConstrainedOptimizer; ...) where T
   @ Optim C:\Users\IRCSB01\.julia\packages\Optim\ZhuZN\src\multivariate\solvers\constrained\fminbox.jl:253
  optimize(::Any, !Matched::Number, !Matched::Number, !Matched::AbstractArray{T}, !Matched::Optim.AbstractConstrainedOptimizer, !Matched::Optim.Options; kwargs...) where T
   @ Optim C:\Users\IRCSB01\.julia\packages\Optim\ZhuZN\src\multivariate\solvers\constrained\fminbox.jl:251
  ...


In [ ]:
fig1 = Figure(size = (800, 600))
ax1 = Axis(fig1[1, 1], title = "Value Functions", xlabel = "Assets", ylabel = "Value")
for j in 1:pars.nz
    lines!(ax1, Kvals, Value[:,j], label = "Shock $j")
end
legend = Legend(fig1[1,2], ax1, "Legend", orientation = :vertical, fontsize = 4)
display(fig1)

fig2 = Figure(size = (800, 600))
ax2 = Axis(fig2[1, 1], title = "Policy Functions", xlabel = "Assets Today", ylabel = "Assets Tomorrow")
for j in 1:pars.nz
    lines!(ax2, Kvals, Policy[:,j], label = "Shock $j")
end
lines!(ax2, Kvals, Kvals, label = "45 Deg Line", color = :black, linestyle = :dash)
legend = Legend(fig2[1,2], ax2, "Legend", orientation = :vertical, fontsize = 4)
display(fig2)